In [1]:
%cd /content/drive/MyDrive/tta/SAR
import torch
import torch.optim as optim
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, ConcatDataset
from torchvision import datasets, transforms, models
from torchvision.io.image import read_image
from torchvision.transforms.functional import normalize, resize, to_pil_image

import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import precision_recall_fscore_support
import copy
import math

import cv2
import numpy as np

!pip install timm

# ..
from backbone import ERM
from adapt_methods import SAR, PseudoLabel, SHOT, T3A, PLClf


/content/drive/MyDrive/tta/SAR
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 7.8 MB/s eta 0:00:00


In [47]:
from pytorch_grad_cam import GradCAM, ScoreCAM
from pytorch_grad_cam import GuidedBackpropReLUModel
from pytorch_grad_cam.utils.image import show_cam_on_image, \
    preprocess_image

In [46]:
!pip install grad-cam
#!pip install torchcam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 20.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for grad-cam: filename=grad_cam-1.5.0-py3-none-any.whl size=38070 sha256=4a928064d0d51593d131ff7663df87b64882e9509849f1e81687416009789419
  Stored in directory: /root/.cache/pip/wheels/5b/e5/3d/8548241d5cffe53ad1476c566a61ad9bf09cc61a9430f09726
Successfully built grad-cam


In [2]:
# install packages
#!pip install torchcam
!unzip "../../lesion_rec/isic2.zip" -d "/content"

Streaming output truncated to the last 5000 lines.
  inflating: /content/ISIC2019_train/hair/ben/ISIC_0024420.jpg  
  inflating: /content/ISIC2019_train/hair/ben/ISIC_0024421.jpg  
  inflating: /content/ISIC2019_train/hair/ben/ISIC_0024424.jpg  
  inflating: /content/ISIC2019_train/hair/ben/ISIC_0024426.jpg  
  inflating: /content/ISIC2019_train/hair/ben/ISIC_0024428.jpg  
  inflating: /content/ISIC2019_train/hair/ben/ISIC_0024430.jpg  
  inflating: /content/ISIC2019_train/hair/ben/ISIC_0024444.jpg  
  inflating: /content/ISIC2019_train/hair/ben/ISIC_0024446.jpg  
  inflating: /content/ISIC2019_train/hair/ben/ISIC_0024451.jpg  
  inflating: /content/ISIC2019_train/hair/ben/ISIC_0024466.jpg  
  inflating: /content/ISIC2019_train/hair/ben/ISIC_0024467.jpg  
  inflating: /content/ISIC2019_train/hair/ben/ISIC_0024473.jpg  
  inflating: /content/ISIC2019_train/hair/ben/ISIC_0024475.jpg  
  inflating: /content/ISIC2019_train/hair/ben/ISIC_0024476.jpg  
  inflating: /content/ISIC2019_train/ha

In [3]:
data_root = "/content/ISIC2019_train"
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        #transforms.RandomResizedCrop(224, scale=(0.7, 1.0)),
        #transforms.RandomHorizontalFlip(),
        #transforms.ColorJitter(0.3, 0.3, 0.3, 0.3),
        #transforms.RandomGrayscale(),
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]),
}

image_datasets = {
    'clean': datasets.ImageFolder(data_root + '/clean', data_transforms['train']),
    'dark_corner': datasets.ImageFolder(data_root + '/dark_corner', data_transforms['train']),
    'gel_bubble': datasets.ImageFolder(data_root + '/gel_bubble', data_transforms['train']),
    'hair': datasets.ImageFolder(data_root + '/hair', data_transforms['train']),
    'ruler': datasets.ImageFolder(data_root + '/ruler', data_transforms['train']),
}

In [29]:
def train_model(model, num_epochs=3):
    device = 'cuda'
    torch.manual_seed(0)
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        model.train()

        running_loss = 0.0
        running_corrects = 0
        step = 0
        for (x, y) in dataloaders['train_set']:
            x = x.to(device)
            y = y.to(device)


            outputs = model.predict(x).to(device)

            criterion = nn.CrossEntropyLoss()
            loss = criterion(outputs, y)

            model.optimizer.zero_grad()
            loss.backward()
            model.optimizer.step()

            running_loss += loss.item() * x.size(0)
            running_corrects += torch.sum(outputs.argmax(1) == y)

            if step % 50 == 0:
              print(step, "/", len(dataloaders['train_set']))
            step += 1

        epoch_loss = running_loss / len(image_datasets['train_set'])
        epoch_acc = running_corrects.double() / len(image_datasets['train_set'])

        print('{} loss: {:.4f}, acc: {:.4f}'.format('train',
                                                    epoch_loss,
                                                    epoch_acc))

    return model

In [25]:
#evaluation
def evaluate(adapt_model, target_class, adapt=False):
  device = 'cuda'
  torch.manual_seed(0)
  total = 0
  correct = 0
  predicted_labels = []
  ground_truth_labels = []

  adapt_model.eval()
  with torch.no_grad():
    for x, y in tqdm(dataloaders[target_class]):
      x = x.to(device)
      y = y.to(device)
      if adapt:
        outputs = adapt_model(x, adapt)
      else:
        outputs = adapt_model(x)
      preds = outputs.argmax(1)
      total += 32
      correct += torch.sum(preds == y)
      #print(outputs, preds, correct)
      # appending
      predicted_labels.extend(preds.cpu().tolist())
      ground_truth_labels.extend(y.cpu().tolist())

  acc = correct/total
  pr,rc, fscore,_ = precision_recall_fscore_support(ground_truth_labels, predicted_labels, average='binary',zero_division=0)
  print(pr, rc, fscore)
  return acc, pr, rc, fscore

In [37]:
# create training set of 4 domains
torch.manual_seed(0)
#domain_list = ['clean','ruler','gel_bubble','hair','dark_corner']
domain_list = ['clean','gel_bubble','hair','dark_corner', 'ruler']
model_types = ['deit', 'resnet50']

for domain in ['hair']:
  print("Testing on domain: ", domain)
  # leave one out
  train_domains = [x for x in domain_list if x != domain]
  print(domain_list)

  train_datasets = [image_datasets[x] for x in train_domains]

  # load train and test sets
  image_datasets['train_set'] = ConcatDataset(train_datasets)
  dataloaders = {}
  dataloaders['train_set'] = torch.utils.data.DataLoader(image_datasets['train_set'],
                                batch_size=32,
                                shuffle=True,
                                num_workers=2)
  dataloaders[domain] = torch.utils.data.DataLoader(image_datasets[domain],
                                batch_size=32,
                                shuffle=True,
                                num_workers=2)

  # define model
  model = ERM('deit').to('cuda')
  model_trained = train_model(model, 5)

  print("Evaluating base model")
  acc,pr,rc,f1 = evaluate(model_trained, domain)
  print(f'Acc.: {acc.item():.3f} + Rc: {rc:.3f} + Pr. {pr:.3f} + fc. {f1:.3f}')

  print("Evaluating adaptation models")

  adapt_methods = {'SAR':SAR,'T3A':T3A, 'SHOT':SHOT, 'PL':PseudoLabel, 'PLClf': PLClf}
  for adapt_method in adapt_methods:
      print(f'Using: {adapt_method}...')
      adapt_model = adapt_methods[adapt_method](model_trained)
      acc,pr,rc,f1 = evaluate(adapt_model, domain, True)
      print(f'{adapt_method} Acc.: {acc.item():.3f} + Rc: {rc:.3f} + Pr. {pr:.3f} + fc. {f1:.3f}')

Testing on domain:  hair
['clean', 'gel_bubble', 'hair', 'dark_corner', 'ruler']


/content/drive/MyDrive/tta/SAR/vision_transformer.py:40: UserWarning: Mapping deprecated model name vit_small_r26_s32_224_in21k to current vit_small_r26_s32_224.augreg_in21k.
  self.network = func(pretrained=True)


Epoch 1/10
----------
0 / 234
50 / 234
100 / 234
150 / 234
200 / 234
train loss: 0.5543, acc: 0.7803
Epoch 2/10
----------
0 / 234
50 / 234
100 / 234
150 / 234
200 / 234
train loss: 0.4932, acc: 0.7846
Epoch 3/10
----------
0 / 234
50 / 234
100 / 234
150 / 234
200 / 234
train loss: 0.4661, acc: 0.7858
Epoch 4/10
----------
0 / 234
50 / 234
100 / 234
150 / 234
200 / 234
train loss: 0.4602, acc: 0.7897
Epoch 5/10
----------
0 / 234
50 / 234
100 / 234
150 / 234
200 / 234
train loss: 0.4654, acc: 0.7892
Epoch 6/10
----------
0 / 234
50 / 234
100 / 234
150 / 234
200 / 234
train loss: 0.4456, acc: 0.7989
Epoch 7/10
----------
0 / 234
50 / 234
100 / 234
150 / 234
200 / 234
train loss: 0.4414, acc: 0.8055
Epoch 8/10
----------
0 / 234
50 / 234
100 / 234
150 / 234
200 / 234
train loss: 0.4386, acc: 0.8061
Epoch 9/10
----------
0 / 234
50 / 234
100 / 234
150 / 234
200 / 234
train loss: 0.4287, acc: 0.8116
Epoch 10/10
----------
0 / 234
50 / 234
100 / 234
150 / 234
200 / 234
train loss: 0.4274, a

100%|██████████| 153/153 [00:24<00:00,  6.17it/s]


0.3527272727272727 0.31442463533225284 0.3324764353041988
Acc.: 0.838 + Rc: 0.314 + Pr. 0.353 + fc. 0.332
Evaluating adaptation models
Using: SAR...


...

0.15492957746478872 0.6418152350081038 0.24960605105578318
SHOT Acc.: 0.511 + Rc: 0.642 + Pr. 0.155 + fc. 0.250
Using: PL...


100%|██████████| 153/153 [00:47<00:00,  3.25it/s]


0.5 0.0016207455429497568 0.003231017770597738
PL Acc.: 0.872 + Rc: 0.002 + Pr. 0.500 + fc. 0.003
Using: PLClf...


100%|██████████| 153/153 [00:45<00:00,  3.36it/s]

0.41530054644808745 0.12317666126418152 0.19
PLClf Acc.: 0.865 + Rc: 0.123 + Pr. 0.415 + fc. 0.190


In [34]:
acc,pr,rc,f1 = evaluate(PLClf(model_trained), 'hair', True)

100%|██████████| 153/153 [00:43<00:00,  3.52it/s]

0.37777777777777777 0.027552674230145867 0.0513595166163142
Acc.: 0.87 + Rc: 0.03 + Pr. 0.38 + fc. 0.05


In [10]:
objct

(tensor(0.8715, device='cuda:0'), 0.0, 0.0, 0.0)

In [7]:
evaluate(model_trained, 'ruler')

100%|██████████| 21/21 [00:05<00:00,  3.83it/s]


(tensor(0.7574, device='cuda:0'), 0.0, 0.0, 0.0)